In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir("/Users/acamara/zav/ragelo/notebooks/LLMJudge/")

In [51]:
from collections import defaultdict
from typing import List, Dict
import json
from dataclasses import dataclass
from ragelo import Query, QueriesDataset
import pickle

@dataclass
class Sample:
    qid: str
    did: str
    query: str
    doc: str
    rel: int

queries = {}
for line in open("data/llm4eval_query_2024.txt"):
    qid, query = line.strip().split("\t")
    queries[qid] = query

corpus = {}
for line in open("data/llm4eval_document_2024.jsonl"):
    data = json.loads(line)
    corpus[data['docid']] = data['doc']

dataset: List[Query] = []
query_ids = {}
qrels = defaultdict(lambda:dict())
# ragelo_dataset = QueriesDataset()

for line in open("data/llm4eval_dev_qrel_2024.txt"):
    qid, _, did, rel = line.strip().split()
    qrels[qid][did] = int(rel)
    # if qid not in ragelo_dataset:
        # ragelo_dataset[qid] = queries[qid]
    dataset.append(
        Sample(
            qid=qid,
            did=did,
            query=queries[qid],
            doc=corpus[did],
            rel=int(rel)
        )
    )
               
    

In [19]:
all_docs = 0
for q in ragelo_dataset:
    all_docs += len(q.retrieved_docs)
pickle.dump(ragelo_dataset, open("ragelo_dev_dataset_all.pkl", "wb"))

In [45]:
import random
random.shuffle(dataset)
dev_dataset = dataset[:200]
scores = defaultdict(lambda: 0)
for d in dev_dataset:
    scores[d.rel]+=1
for s in sorted(scores.keys()):
    print (s, scores[s]/len(dev_dataset))

0 0.66
1 0.17
2 0.075
3 0.095


In [52]:
queries[q.qid]

'What vitamins can help with pain in the arm after a blood test?'

In [55]:
ragelo_dataset = QueriesDataset()
for q in dev_dataset:
    if q.qid not in ragelo_dataset:
        ragelo_dataset[q.qid] = queries[q.qid]
    ragelo_dataset.add_retrieved_doc(q.qid, doc=q.doc, doc_id = q.did)

In [56]:
ragelo_dataset.save("/Users/acamara/zav/ragelo/notebooks/LLMJudge/ragelo_dev_sample.pkl")

In [57]:
ragelo_dataset = QueriesDataset.load("/Users/acamara/zav/ragelo/notebooks/LLMJudge/ragelo_dev_sample.pkl")

In [18]:
scores = defaultdict(lambda: 0)
for d in dataset:
    scores[d.rel]+=1
for s in sorted(scores.keys()):
    print (s, scores[s]/len(dataset))

0 0.6248106842902382
1 0.19317086603331957
2 0.08605259534627564
3 0.09596585433016659


In [58]:
from ragelo import get_llm_provider, get_retrieval_evaluator
from ragelo.types.configurations import CustomPromptEvaluatorConfig
llm_provider = get_llm_provider("openai", model="gpt-4o")

In [59]:
retrieval_evaluator_config = CustomPromptEvaluatorConfig(
    prompt="""
You are an expert document annotator. Your job is to evaluate whether a document contains relevant information to answer a user's question.

Please act as an impartial relevance annotator for a search engine. Your goal is to evaluate the relevancy of the documents given a user question.

You should write one sentence explaining why the document is relevant or not for the user question. A document can be:
    - Not relevant: The passage has nothing to do with the query.
    - Related: The passage seems related to the user's question, but does not answer it.
    - Somewhat Relevant: The passage has some answer for the query, but the answer may be a bit unclear, or hidden amongst extraneous information.
    - Very relevant: The passage is dedicated to the query and contains the exact answer.
    [user question]
    {query}

    [document content]
    {document}""", # noqa: E501
    # scoring_keys=["relevance"],
    answer_format="text"
)
retrieval_evaluator = get_retrieval_evaluator(config=retrieval_evaluator_config, llm_provider=llm_provider)

In [62]:
ragelo_dataset.batch_evaluate(retrieval_evaluator)
ragelo_dataset.save("/Users/acamara/zav/ragelo/notebooks/LLMJudge/ragelo_dev_sample.pkl")
ragelo_dataset = QueriesDataset.load("/Users/acamara/zav/ragelo/notebooks/LLMJudge/ragelo_dev_sample.pkl")

AttributeError: 'str' object has no attribute 'retrieved_docs'

In [69]:
def parse_reasoner_to_int(raw_answer:str) -> int:
    ans = raw_answer.strip().replace("- ", "")
    if ans.lower().startswith("not relevant"):
        return 0
    if ans.lower().startswith("related"):
        return 1
    elif ans.lower().startswith("somewhat relevant"):
        return 2
    elif ans.lower().startswith("very relevant"):
        return 3
    else:
        raise ValueError(f"No idea how to parse '{raw_answer}'")
    

for q in ragelo_dataset:
    for d in q.retrieved_docs:
        d.evaluation.answer = parse_reasoner_to_int(d.evaluation.raw_answer)
        

In [71]:
from sklearn.metrics import root_mean_squared_error, mean_squared_error
import numpy as np
labels = []
est = []
for q in ragelo_dataset:
    for d in q.retrieved_docs:
        est.append(d.evaluation.answer)
        labels.append(qrels[q.qid][d.did])
print(f"RMSE: {root_mean_squared_error(labels, est)}")
print(f"MSE:  {mean_squared_error(labels, est)}")
np.median(labels), np.median(est)

RMSE: 0.9327379053088815
MSE:  0.87


(0.0, 0.0)